In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0-beta1


The custom losses and metrics we defined earlier were all based on the labels and the
predictions (and optionally sample weights). However, you will occasionally want to
define losses based on other parts of your model, such as the weights or activations of
its hidden layers. This may be useful for regularization purposes, or to monitor some
internal aspect of your model.

To define a custom loss based on model internals, just compute it based on any part
of the model you want, then pass the result to the ```add_loss()``` method. 

For example, the following custom model represents a standard MLP regressor with 5 hidden 
layers, except it also implements a reconstruction loss (we add an extra Dense
layer on top of the last hidden layer, and its role is to try to reconstruct the inputs of
the model. Since the reconstruction must have the same shape as the model’s inputs,
we need to create this Dense layer in the ```build()``` method to have access to the shape
of the inputs. In the ```call()``` method, we compute both the regular output of the MLP,
plus the output of the reconstruction layer. We then compute the mean squared difference 
between the reconstructions and the inputs, and we add this value (times 0.05) to the 
model’s list of losses by calling ```add_loss()```. 

During training, tf.keras will add this loss to the main loss (which is why we 
scaled down the reconstruction loss,to ensure the main loss dominates). 
As a result, the model will be forced to preserve as much information as possible 
through the hidden layers, even information that is not directly useful for the regression
task itself. In practice, this loss sometimes improves generalization; 
it is a regularization loss:

In [4]:
class ReconstructingRegressor(tf.keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(30, activation="selu",
                                          kernel_initializer="lecun_normal")
                       for _ in range(5)]
        self.out = tf.keras.layers.Dense(output_dim)
        self.reconstruction_mean = tf.keras.metrics.Mean(name="reconstruction_error")

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = tf.keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)

    @tf.function
    def call(self, inputs, training=None):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * reconstruction_loss)
        if training:
            result = self.reconstruction_mean(recon_loss)
            self.add_metric(result)
        return self.out(Z)

model = ReconstructingRegressor(1)
# model.build(tf.TensorShape([None, 8]))       # <= Fails if this line is removed
model.compile(loss="mse", optimizer="nadam")

Now during the training, it will show both 'mse' loss and reconstruction_loss
```
history = model.fit(X, y, epochs=2)

Epoch 1/5
11610/11610 [=============] [...] loss: 4.3092 - reconstruction_error: 1.7360
Epoch 2/5
11610/11610 [=============] [...] loss: 1.1232 - reconstruction_error: 0.8964
[...]
```